#Libraries

In [ ]:
!pip install torch_geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.2/616.2 KB 13.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.0-py3-none-any.whl size=909897 sha256=58e38dec9bedfc57bdf197410042fd56cc4fc864b382770a5ea7ccd73014b13f
  Stored in directory: /root/.cache/pip/wheels/cd/7d/6b/17150450b80b4a3656a84330e22709ccd8dc0f8f4773ba4133
Successfully built torch_geometric


In [ ]:
!pip install torch_sparse

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 KB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch_sparse: filename=torch_sparse-0.6.17-cp39-cp39-linux_x86_64.whl size=1082789 sha256=eac84f76ebe948917eabe3c4e5cb340dc2d4e31dd0396836eff0ccb9ab779319
  Stored in directory: /root/.cache/pip/wheels/f8/43/54/bcb8acdd1109bd1e4c71106747af298d0315cdf3f090b2ae43
Successfully built torch_sparse


In [ ]:
!pip install torch_scatter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 KB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch_scatter: filename=torch_scatter-2.1.1-cp39-cp39-linux_x86_64.whl size=484407 sha256=b0d6fc624874e82c002fef86d4d65dcd6fe6134b866a2fecd2cf268f1f1e2ae6
  Stored in directory: /root/.cache/pip/wheels/d5/0c/18/11b4cf31446c5d460543b0fff930fcac3a3f8a785e5c73fb15
Successfully built torch_scatter


In [ ]:
!pip install utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install gnn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from collections import defaultdict
from pickle import FALSE
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn.modules.activation import PReLU
from torch_geometric.nn import GCNConv, GATConv,RGCNConv
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.inits import glorot, uniform
from torch_geometric.utils import softmax
import math

from torch_sparse.tensor import to
from utils import *
from torch_sparse import SparseTensor, set_diag
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax, degree, to_undirected
from torch.nn import Sequential, Linear, ReLU, Dropout
from torch_scatter import scatter
import numpy as np

In [ ]:
from matplotlib.pyplot import delaxes
import random
import os
import pandas as pd
import time
import scipy.sparse as sparse

In [ ]:
from os import times
from tqdm.utils import SimpleTextIOWrapper

from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

# from gnn import RiskGNN
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import recall_score as rec
from sklearn.metrics import precision_score as pre
from sklearn.metrics import f1_score as f1
from sklearn.metrics import roc_auc_score as roc

import argparse

# **Functions**

#utils

In [ ]:
def  set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    # os.environ['CUDA_LAUNCH_BLOCKING'] = str(1)
    # os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.enabled = False
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms(True)

In [ ]:
class Classifier(nn.Module):
    def __init__(self, n_hid, n_out):
        super(Classifier, self).__init__()
        self.n_hid = n_hid
        self.n_out = n_out
        self.linear = nn.Linear(n_hid, n_out)

    def forward(self, x):
        tx = self.linear(x)
        return torch.log_softmax(tx.squeeze(), dim=-1)

    def __repr__(self):
        return '{}(n_hid={}, n_out={})'.format(
            self.__class__.__name__, self.n_hid, self.n_out)

Hyper Graph

In [ ]:
#refer to https://github.com/iMoonLab/THU-HyperG/blob/master/hyperg/hyperg.py
class HyperG:
    def __init__(self, H, X=None, w=None):
        """ Initial the incident matrix, node feature matrix and hyperedge weight vector of hypergraph
        :param H: scipy coo_matrix of shape (n_nodes, n_edges)
        :param X: numpy array of shape (n_nodes, n_features)
        :param w: numpy array of shape (n_edges,)
        """
        assert sparse.issparse(H)
        assert H.ndim == 2

        self._H = H
        self._n_nodes = self._H.shape[0]
        self._n_edges = self._H.shape[1]

        if X is not None:
            assert isinstance(X, np.ndarray) and X.ndim == 2
            self._X = X
        else:
            self._X = None

        if w is not None:
            self.w = w.reshape(-1)
            assert self.w.shape[0] == self._n_edges
        else:
            self.w = np.ones(self._n_edges)

        self._DE = None
        self._DV = None
        self._INVDE = None
        self._DV2 = None
        self._THETA = None
        self._L = None

    def num_edges(self):
        return self._n_edges

    def num_nodes(self):
        return self._n_nodes

    def incident_matrix(self):
        return self._H

    def hyperedge_weights(self):
        return self.w

    def node_features(self):
        return self._X

    def node_degrees(self):
        if self._DV is None:
            H = self._H.tocsr()
            dv = H.dot(self.w.reshape(-1, 1)).reshape(-1)
            self._DV = sparse.diags(dv, shape=(self._n_nodes, self._n_nodes))
        return self._DV

    def edge_degrees(self):
        if self._DE is None:
            H = self._H.tocsr()
            de = H.sum(axis=0).A.reshape(-1)
            self._DE = sparse.diags(de, shape=(self._n_edges, self._n_edges))
        return self._DE

    def inv_edge_degrees(self):
        if self._INVDE is None:
            self.edge_degrees()
            inv_de = np.power(self._DE.data.reshape(-1), -1.)
            self._INVDE = sparse.diags(inv_de, shape=(self._n_edges, self._n_edges))
        return self._INVDE

    def inv_square_node_degrees(self):
        if self._DV2 is None:
            self.node_degrees()
            dv2 = np.power(self._DV.data.reshape(-1)+1e-6, -0.5)
            self._DV2 = sparse.diags(dv2, shape=(self._n_nodes, self._n_nodes))
        return self._DV2

    def theta_matrix(self):
        if self._THETA is None:
            self.inv_square_node_degrees()
            self.inv_edge_degrees()

            W = sparse.diags(self.w)
            self._THETA = self._DV2.dot(self._H).dot(W).dot(self._INVDE).dot(self._H.T).dot(self._DV2)

        return self._THETA

    def laplacian(self):
        if self._L is None:
            self.theta_matrix()
            self._L = sparse.eye(self._n_nodes) - self._THETA
        return self._L

    def update_hyedge_weights(self, w):
        assert isinstance(w, (np.ndarray, list)), \
            "The hyperedge array should be a numpy.ndarray or list"

        self.w = np.array(w).reshape(-1)
        assert w.shape[0] == self._n_edges

        self._DV = None
        self._DV2 = None
        self._THETA = None
        self._L = None

    def update_incident_matrix(self, H):
        assert sparse.issparse(H)
        assert H.ndim == 2
        assert H.shape[0] == self._n_nodes
        assert H.shape[1] == self._n_edges

        # TODO: reset hyperedge weights?

        self._H = H
        self._DE = None
        self._DV = None
        self._INVDE = None
        self._DV2 = None
        self._THETA = None
        self._L = None

In [ ]:
def print_log(message):
    """
    :param message: str,
    :return:
    """
    print("[{}] {}".format(time.strftime("%Y-%m-%d %X", time.localtime()), message))

In [ ]:
def gen_attribute_hg(n_nodes, attr_dict, X=None):
    """
    :param attr_dict: dict, eg. {'attri_1': [node_idx_1, node_idx_1, ...], 'attri_2':[...]} (zero-based indexing)
    :param n_nodes: int,
    :param X: numpy array, shape = (n_samples, n_features) (optional)
    :return: instance of HyperG
    """

    if X is not None:
        assert n_nodes == X.shape[0]

    n_edges = len(attr_dict)
    node_idx = []
    edge_idx = []

    for idx, attr in enumerate(attr_dict):
        nodes = sorted(attr_dict[attr])
        node_idx.extend(nodes)
        edge_idx.extend([idx] * len(nodes))

    node_idx = np.asarray(node_idx)
    edge_idx = np.asarray(edge_idx)
    values = np.ones(node_idx.shape[0])

    H = sparse.coo_matrix((values, (node_idx, edge_idx)), shape=(n_nodes, n_edges))
    return HyperG(H, X=X)

In [ ]:
def scipy_sparse_mat_to_torch_sparse_tensor(sparse_mx):
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [ ]:
#refer to https://github.com/alge24/DyGNN/blob/b161555a5df69bd3fa9cc3ae5d4f5cd65ebe3a0f/decayer.py
class Decayer(nn.Module):
    def __init__(self, w1=0.01,w2=0.1, decay_method='rev'):
    # def __init__(self, w1=100,w2=200, decay_method='rev'):
        super(Decayer,self).__init__()
        self.decay_method = decay_method
        self.w1 = w1
        self.w2=w2

    def exponetial_decay(self, w, delta_t):
        return torch.exp(-w*delta_t)
    def log_decay(self, w, delta_t):
        return 1/torch.log(2.7183 + w*delta_t)
    def rev_decay(self, w, delta_t):
        return 1/(1 + w*delta_t)

    def forward(self,delta_t):
        seq=torch.zeros_like(delta_t)

        idx1=(delta_t<=24)
        idx2=(delta_t>24)

        # # print(delta_t)
        if self.decay_method == 'exp':
            seq[idx1]=self.exponetial_decay(self.w1,delta_t[idx1])
            seq[idx2]=self.exponetial_decay(self.w2,delta_t[idx2])
        elif self.decay_method == 'log':
            seq[idx1]=self.log_decay(self.w1,delta_t[idx1])
            seq[idx2]=self.log_decay(self.w2,delta_t[idx2])
        elif self.decay_method == 'rev':
            seq[idx1]=self.rev_decay(self.w1,delta_t[idx1])
            seq[idx2]=self.rev_decay(self.w2,delta_t[idx2])

        else:
            seq[idx1]=self.exponetial_decay(delta_t[idx1])
            seq[idx2]=self.exponetial_decay(delta_t[idx2])
        # print(seq,"----")

        return seq

In [ ]:
def initializae_company_info(risk_data,company_attr,company_num,cause_type_num,court_type,category,idx=None):
    if idx:
        idx_dict={index:ser for ser,index in enumerate(idx)}
    company_risk=np.zeros((company_num,cause_type_num+court_type+category+1))
    for index in risk_data:
        risk_info=risk_data[index]
        cause_info=[0 for i in range(cause_type_num)]
        court_info=[0 for i in range(court_type)]
        res_info=[0 for i in range(category)]
        time_info=[]
        for i in range(len(risk_info)):
            justify=risk_info[i]
            cause=justify[0]
            court=justify[1]
            res=justify[2]
            time=justify[3]
            cause_info[cause]+=1
            court_info[court]+=1
            res_info[res]+=1
            time_info+=[time]
        time_ave=[np.average(time_info)]
        if idx:
            company_risk[idx_dict[index]]=np.concatenate((cause_info,court_info,res_info,time_ave),axis=0)
        else:
            company_risk[index]=np.concatenate((cause_info,court_info,res_info,time_ave),axis=0)
    company_attr=np.array(company_attr)
    company_info=np.concatenate((company_attr,company_risk),axis=1)
    return company_info

#Creating Heterogeneous Graphs

In [ ]:
class HeteGNN(MessagePassing):
    def __init__(self, input_dim,output_dim,rel_num,negative_slope=0.2,num_company_rel=7,num_person_rel=3,
    aggr = "add", flow= "source_to_target", node_dim = -2):
        super(HeteGNN,self).__init__(aggr=aggr, flow=flow, node_dim=node_dim)
        self.input_dim=input_dim
        self.output_dim=output_dim
        self.rel_num=rel_num
        self.negative_slope=negative_slope

        self.proj_com=nn.Linear(input_dim,output_dim,bias=False)
        self.proj_per=nn.Linear(input_dim,output_dim,bias=False)

        self.ck_linears   = nn.ModuleList()
        self.cq_linears   = nn.ModuleList()

        for t in range(rel_num):
            self.ck_linears.append(nn.Linear(output_dim,   output_dim))
            self.cq_linears.append(nn.Linear(output_dim,   output_dim))

        self.cv_linear=nn.Linear(output_dim,   output_dim)
        self.crelation_pri   = nn.Parameter(torch.ones(rel_num))

        self.rel_wi=nn.ModuleList()
        for i in range(rel_num):
            if i in [6,7,8,9]:
                self.rel_wi.append(nn.Linear(output_dim,output_dim,bias=False))
            else:
                self.rel_wi.append(nn.Linear(output_dim*2,output_dim,bias=False))


        self.skip = nn.Parameter(torch.ones(1))
        self.bn=nn.BatchNorm1d(output_dim)

    def forward(self,company_emb,person_emb,edge_index,edge_type,edge_weight,company_num,person_num):
        company_emb=self.proj_com(company_emb)
        person_emb=self.proj_per(person_emb)
        emb=torch.cat((company_emb,person_emb),dim=0)
        emb=self.bn(emb)
        edge_index= torch.LongTensor(edge_index).transpose(0,1)

        edge_type=torch.LongTensor(edge_type)
        edge_weight=torch.FloatTensor(edge_weight).unsqueeze(1)

        rs_list=[]
        rel_type=[]

        for i in range(self.rel_num):
            mask = (edge_type == i)
            sub_edge_index = edge_index[:, mask]
            sub_edge_weight=edge_weight[mask]
            if mask.sum() !=0:
                rs=F.leaky_relu((self.propagate(sub_edge_index, x=emb,edge_weight=sub_edge_weight,edge_type=i)),self.negative_slope)
                rs_list+=[rs]
                rel_type+=[i]
        com_att=[]

        for ser,i in enumerate(rel_type):

            rel_emb=rs_list[ser]
            q_mat = self.cq_linears[i](emb)
            k_mat = self.ck_linears[i](rel_emb)
            res_att = ((q_mat * k_mat).sum(dim=-1) * self.crelation_pri[i] / math.sqrt(self.output_dim)).unsqueeze(1)

            com_att+=[res_att]

        com_attscore=torch.cat(com_att,dim=1)
        com_attscore=F.softmax(com_attscore,dim=1)
        res=0
        for i in range(len(com_att)):
            res+= com_attscore[:,i].unsqueeze(1) * self.cv_linear(rs_list[i])
        alpha=torch.sigmoid(self.skip)
        res=(res+alpha*F.gelu(emb))
        res_c,res_p=res[:company_num],res[company_num:]

        return res_c,res_p

    def message(self,edge_index, x_i,x_j, edge_weight, edge_type):
        if torch.sum(edge_weight)!=edge_index.shape[1]:
            x_j=self.rel_wi[edge_type](x_j)
            edge_weight=softmax(edge_weight,edge_index[1])
            rs=x_j*edge_weight
        else:
            node_f = torch.cat((x_i, x_j), 1)                                       #nx2d
            temp = self.rel_wi[edge_type](node_f).to(x_i.device)      #nx1

            alpha=softmax(temp,edge_index[1])

            rs=x_j*alpha
        return rs

    def update(self, inputs):
        return super().update(inputs)

#Hyper GNN

In [ ]:
class HyperGNN(nn.Module):
    def __init__(self,input_dim,output_dim,hyper_edge_num=3,num_layer=1,negative_slope=0.2):
        super(HyperGNN,self).__init__()
        self.negative_slope=negative_slope
        
        self.proj=nn.Linear(input_dim,output_dim,bias=False)
       
        self.alpha=nn.Parameter(torch.ones(hyper_edge_num,1))
        
        glorot(self.alpha)

    def forward(self,company_emb,hyp_graph):
        outlist=[]
        for i in range(len(hyp_graph)):
            laplacian=scipy_sparse_mat_to_torch_sparse_tensor(hyp_graph[i].laplacian())
            rs= laplacian@self.proj(company_emb)
            outlist+=[rs]
           
        res=0
       
        alpha=torch.sigmoid(self.alpha)
        print("alpha = ", alpha)
        print("Outlist len = ", len(outlist))
        print("Outlist 0 len = ", len(outlist[0]))
        print("Outlist 0 1 len = ", len(outlist[0][1]))

        print("Outlist 1 len = ", len(outlist[1]))
        print("Outlist 2 len = ", len(outlist[2]))

        for i in range(len(outlist)):
            #print("RES ", i, "= ", res)
            res+=outlist[i]*alpha[i]
            
        print("res len = ", len(res))
        print("res 0 len =", len(res[0]))
        print("res 5 len =", len(res[5]))
        print("res 10 len =", len(res[10]))
        print("res 0 = ", res[0])
        return res

#Risk Analysis

In [ ]:
class RiskInfo(nn.Module):
    def __init__(self,input_dim,company_num,cause_type_num,court_type_num,res_num,time_label_num):
        super(RiskInfo,self).__init__()
        self.input_dim=input_dim
        self.company_num=company_num
        self.time_lable_num=2


        self.ca_emb=nn.Embedding(cause_type_num,12)
        self.court_emb=nn.Embedding(court_type_num,4)
        self.cate_emb=nn.Embedding(res_num,4)
        self.lstm_hidden=20

        self.proj=nn.Linear(20,20,bias=False)


        self.time_decay=Decayer()


    def forward(self,risk_data):

        com_emb=torch.zeros((self.company_num,self.lstm_hidden))

        for index in risk_data:
            cause=self.ca_emb(torch.LongTensor(risk_data[index])[:,0]) 
            court=self.court_emb(torch.LongTensor(risk_data[index])[:,1])
            cate=self.cate_emb(torch.LongTensor(risk_data[index])[:,2])
            risk=torch.cat((cause,court,cate),dim=1)

            time_interval=torch.FloatTensor(risk_data[index])[:,3]

            time_interval=self.time_decay(time_interval).unsqueeze(1)

            risk=self.proj(time_interval*risk)

            com_emb[index]=risk.sum(0)
        return com_emb

#Risk GNN

In [ ]:
class RiskGNN(nn.Module):
    def __init__(self,input_dim,output_dim,
    company_num,person_num,rel_num,cause_type_num,
     device,com_initial_emb,person_initial_emb,
     court_type_num=4,category_num=4,time_label_num=5,num_heads=1,dropout=0.2,norm=True,
     ):
        super(RiskGNN,self).__init__()
        self.input_dim=input_dim
        self.output_dim=output_dim
        self.company_num=company_num
        self.person_num=person_num
        self.rel_num=rel_num
        self.cause_type=cause_type_num
        self.device=device
        self.court_type=court_type_num
        self.category_=category_num
        self.num_heads=num_heads
        self.dropout=dropout
        self.norm=norm
        self.company_emb=torch.FloatTensor(com_initial_emb)
        self.person_emb=torch.FloatTensor(person_initial_emb)

        self.riskinfo=RiskInfo(input_dim,company_num,cause_type_num,court_type_num,category_num,time_label_num=time_label_num)
        self.hypergnn=HyperGNN(input_dim,output_dim,num_layer=1)
        self.hetegnn=nn.ModuleList()
        for i in range(5):
            if i==0:
                self.hetegnn.append(HeteGNN(input_dim,output_dim,rel_num))
            else:
                self.hetegnn.append(HeteGNN(output_dim,output_dim,rel_num))

        self.company_proj=nn.Linear(32,input_dim,bias=False)
        self.person_proj=nn.Linear(32,input_dim,bias=False)

        self.risk_proj=nn.Linear(input_dim+23,input_dim,bias=False)
        self.info_proj=nn.Linear(output_dim,output_dim,bias=False)

        self.final_proj=nn.Sequential(nn.Linear(input_dim,output_dim,bias=False),nn.ReLU(),nn.Linear(output_dim,output_dim,bias=False))
        self.alpha=torch.ones((1))


    # risk data: dict-->{company_index:[[cause type, court type, category, time(months),time_label],...] }
    # company attribute information: np.array()-->[[register_captial, paid_captial, set up time(months)]]
    # graph: edge index:[sour,tar].T -->2xN; edge type: [,,...,] -->N; edge weight:[,,...,]-->N
    # hyper graph: dict:{industry:{ind1:[...],ind2:[...],...},area:{area1:[...],area2:[...],...},qualify:{qua1:[...],qua2:[...],...}}
    def forward(self,risk_data,company_attr,hete_graph,hyp_graph,idx,x):
        company_emb=self.company_proj(self.company_emb)
        person_emb=self.person_proj(self.person_emb)
        company_basic_info=torch.zeros((self.company_num,len(company_attr[0])))
       
        company_basic_info[idx]=torch.Tensor(company_attr)
        
        company_emb=torch.cat((company_emb,company_basic_info),dim=1)
        risk_info=self.riskinfo(risk_data)
        company_emb_info=self.risk_proj(torch.cat((company_emb,risk_info),dim=1))

        company_emb_hyper=self.hypergnn(company_emb_info,hyp_graph)

        edge_index,edge_type,edge_weight=hete_graph

        for i in range(5):
            if i==0:
                company_emb_hete,person_emb=self.hetegnn[i](company_emb_info,person_emb,edge_index,edge_type,edge_weight,self.company_num,self.person_num)
            else:
                company_emb_hete,person_emb=self.hetegnn[i](company_emb_hete,person_emb,edge_index,edge_type,edge_weight,self.company_num,self.person_num)
        company_emb_final=self.info_proj(company_emb_hyper+company_emb_hete)

        alpha=torch.sigmoid(self.alpha)
        company_emb_final=alpha*F.gelu(company_emb_final)+(1-alpha)*self.final_proj(company_emb_info)

        return company_emb_final[idx]

#Model Training

In [ ]:
parser = argparse.ArgumentParser(description='Training GNN')

Dataset arguments

In [ ]:
parser.add_argument('--data_dir', type=str, default='./data', help='The address of preprocessed graph.')
parser.add_argument('--model_dir', type=str, default='.\model_save', help='The address for storing the models and optimization results.')
parser.add_argument('--cuda', type=int, default=0, help='Avaiable GPU ID')

_StoreAction(option_strings=['--cuda'], dest='cuda', nargs=None, const=None, default=0, type=<class 'int'>, choices=None, required=False, help='Avaiable GPU ID', metavar=None)

Model arguments

In [ ]:
parser.add_argument('--conv_name', type=str, default='riskgnn', choices=['riskgnn'], help='The name of GNN filter.')
parser.add_argument('--input_dim', type=int, default=16, help='Number of input dimension')
parser.add_argument('--output_dim', type=int, default=12, help='Number of output dimension')
parser.add_argument('--n_heads', type=int, default=1, help='Number of attention head')
parser.add_argument('--n_layers', type=int, default=1, help='Number of HeteGAT layers')
parser.add_argument('--dropout', type=float, default=0.2, help='Dropout ratio')

_StoreAction(option_strings=['--dropout'], dest='dropout', nargs=None, const=None, default=0.2, type=<class 'float'>, choices=None, required=False, help='Dropout ratio', metavar=None)

Optimization arguments

In [ ]:
parser.add_argument('--optimizer', type=str, default='adam', choices=['adamw', 'adam', 'sgd', 'adagrad'], help='optimizer to use.')

_StoreAction(option_strings=['--optimizer'], dest='optimizer', nargs=None, const=None, default='adam', type=<class 'str'>, choices=['adamw', 'adam', 'sgd', 'adagrad'], required=False, help='optimizer to use.', metavar=None)

In [ ]:
parser.add_argument('--n_epoch', type=int, default=10, help='Number of epoch to run')

_StoreAction(option_strings=['--n_epoch'], dest='n_epoch', nargs=None, const=None, default=10, type=<class 'int'>, choices=None, required=False, help='Number of epoch to run', metavar=None)

In [ ]:
parser.add_argument('--clip', type=float, default=0.25, help='Gradient Norm Clipping')

_StoreAction(option_strings=['--clip'], dest='clip', nargs=None, const=None, default=0.25, type=<class 'float'>, choices=None, required=False, help='Gradient Norm Clipping', metavar=None)

In [ ]:
parser.add_argument('--weight_decay', type=float, default=1e-4, help='weight decay of adamw ')

_StoreAction(option_strings=['--weight_decay'], dest='weight_decay', nargs=None, const=None, default=0.0001, type=<class 'float'>, choices=None, required=False, help='weight decay of adamw ', metavar=None)

In [ ]:
#args = parser.parse_args()

In [ ]:
args, unknown = parser.parse_known_args()

In [ ]:
device=torch.device("cpu")
set_random_seed(14)

criterion = torch.nn.CrossEntropyLoss()

In [ ]:
total_company_num=3976
person_num=2405
court_type=4
category=4
time_label_num=5

train_company_num=2816
valid_company_num=721
test_company_num=491

rel_num=12
cause_type_num=11

Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_data=pd.read_pickle('/content/drive/MyDrive/GNN/train_data.pkl')
valid_data=pd.read_pickle('/content/drive/MyDrive/GNN/validate_data.pkl')
test_data=pd.read_pickle('/content/drive/MyDrive/GNN/test_data.pkl')
split_data_idx=pd.read_pickle('/content/drive/MyDrive/GNN/split_data_idx.pkl')
com_initial_emb,person_initial_emb=pd.read_pickle('/content/drive/MyDrive/GNN/meta_emb.pkl')

In [ ]:
train_risk_data,train_company_attr,train_hete_graph,train_hyp_graph,train_label=train_data
valid_risk_data,valid_company_attr,valid_hete_graph,valid_hyp_graph,valid_label=valid_data
test_risk_data,test_company_attr,test_hete_graph,test_hyp_graph,test_label=test_data
train_idx,valid_idx,test_idx=split_data_idx

In [ ]:
x_train=initializae_company_info(train_risk_data,train_company_attr,train_company_num,cause_type_num,court_type,category,train_idx)
x_valid=initializae_company_info(valid_risk_data,valid_company_attr,valid_company_num,cause_type_num,court_type,category,valid_idx)
x_test=initializae_company_info(test_risk_data,test_company_attr,test_company_num,cause_type_num,court_type,category,test_idx)

Graph Neural Network

In [ ]:
gnn=RiskGNN(args.input_dim,args.output_dim, total_company_num,person_num,rel_num, cause_type_num, device,com_initial_emb,person_initial_emb, court_type,category,time_label_num, num_heads=1,dropout=0.2,norm=True)

In [ ]:
classifier = Classifier(args.output_dim, 2).to(device)
model = nn.Sequential(gnn, classifier)

In [ ]:
if args.optimizer == 'adamw':
    optimizer = torch.optim.AdamW(model.parameters(),weight_decay=args.weight_decay)
elif args.optimizer == 'adam':
    optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
elif args.optimizer == 'sgd':
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
elif args.optimizer == 'adagrad':
    optimizer = torch.optim.Adagrad(model.parameters())

In [ ]:
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 20, eta_min=1e-6)

In [ ]:
['industry', 'area', 'qualify']
train_hyp=[]
for i in ['industry', 'area', 'qualify']:
    train_hyp+=[gen_attribute_hg(total_company_num, train_hyp_graph[i], X=None)]
valid_hyp=[]
for i in ['industry', 'area', 'qualify']:
    valid_hyp+=[gen_attribute_hg(total_company_num, valid_hyp_graph[i], X=None)]
test_hyp=[]
for i in ['industry', 'area', 'qualify']:
    test_hyp+=[gen_attribute_hg(total_company_num, test_hyp_graph[i], X=None)]

In [ ]:
def test():
    best_model = torch.load('./model_save/%s.pkl'%(args.conv_name))
    best_model.eval()
    gnn, classifier = best_model
    with torch.no_grad():
        company_emb=gnn.forward(test_risk_data,test_company_attr,test_hete_graph,test_hyp,test_idx,x_test)

        res = classifier.forward(company_emb)

        pred=res.argmax(dim=1)
        ac=acc(test_label,pred)
        pr=pre(test_label,pred)
        re=rec(test_label,pred)
        f=f1(test_label,pred)
        rc=roc(test_label,res[:,1])

        print('Best Test Acc: %.4f Best Test Pre: %.4f Best Test Recall: %.4f Best Test F1: %.4f Best Test ROC: %.4f' % (ac,pr,re,f,rc))
        print(sum(pred))

In [ ]:
best_acc=0
best_f1=0

In [ ]:
for epoch in np.arange(args.n_epoch):

    st=time.time()

    '''
        Train 
    '''
    model.train()
    train_losses = []
    # torch.cuda.empty_cache()
    company_emb=gnn.forward(train_risk_data,train_company_attr,train_hete_graph,train_hyp,train_idx,x_train)

    
    res = classifier.forward(company_emb)

    loss = criterion(res, torch.LongTensor(train_label))
    optimizer.zero_grad()
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
    optimizer.step()

    train_losses += [loss.cpu().detach().tolist()]
    # train_step += 1
    scheduler.step()
    del res, loss

    '''
        Valid 
    '''
    model.eval()
    with torch.no_grad():
        company_emb=gnn.forward(valid_risk_data,valid_company_attr,valid_hete_graph,valid_hyp,valid_idx,x_valid)

        res = classifier.forward(company_emb)
        loss = criterion(res,torch.LongTensor(valid_label) )

        pred=res.argmax(dim=1)
        ac=acc(valid_label,pred)
        pr=pre(valid_label,pred)
        re=rec(valid_label,pred)
        f=f1(valid_label,pred)
        rc=roc(valid_label,res[:,1])

        if ac > best_acc and f>best_f1:
            best_acc = ac
            best_f1=f
            torch.save(model, '/content/drive/MyDrive/GNN/%s.pkl'%(args.conv_name))  # args.conv_name = riskgnn

            print('UPDATE!!!')


        et = time.time()
        print(("Epoch: %d (%.1fs)  LR: %.5f Train Loss: %.2f  Valid Loss: %.2f  Valid Acc: %.4f Valid Pre: %.4f  Valid Recall: %.4f Valid F1: %.4f  Valid Roc: %.4f"  ) % \
              (epoch, (et - st), optimizer.param_groups[0]['lr'], np.average(train_losses), \
               loss.cpu().detach().tolist(), ac,pr,re,f,rc))

        del res, loss

        if epoch+1==args.n_epoch:
            company_emb=gnn.forward(test_risk_data,test_company_attr,test_hete_graph,test_hyp,test_idx,x_test)

            res = classifier.forward(company_emb)

            pred=res.argmax(dim=1)
            ac=acc(test_label,pred)
            pr=pre(test_label,pred)
            re=rec(test_label,pred)
            f=f1(test_label,pred)
            rc=roc(test_label,res[:,1])
            
            print('Last Test Acc: %.4f Last Test Pre: %.4f Last Test Recall: %.4f Last Test F1: %.4f Last Test ROC: %.4f' % (ac,pr,re,f,rc))

alpha =  tensor([[0.2254],
        [0.3951],
        [0.5730]], grad_fn=<SigmoidBackward0>)
Outlist len =  3
Outlist 0 len =  3976
Outlist 0 1 len =  12
Outlist 1 len =  3976
Outlist 2 len =  3976
res len =  3976
res 0 len = 12
res 5 len = 12
res 10 len = 12
res 0 =  tensor([-359.7084, -980.8617,  185.2680,  424.3913, -132.3065,  693.8123,
        -372.6189,  128.8235, -152.4756, -817.1254,  710.8715,  -49.8491],
       grad_fn=<SelectBackward0>)
alpha =  tensor([[0.2237],
        [0.3930],
        [0.5712]])
Outlist len =  3
Outlist 0 len =  3976
Outlist 0 1 len =  12
Outlist 1 len =  3976
Outlist 2 len =  3976
res len =  3976
res 0 len = 12
res 5 len = 12
res 10 len = 12
res 0 =  tensor([ 0.0280,  0.0434,  0.0392, -0.0422,  0.0026, -0.0114,  0.0458,  0.0021,
        -0.0359,  0.0322,  0.0212,  0.0420])
UPDATE!!!
Epoch: 0 (1.7s)  LR: 0.00976 Train Loss: 4.80  Valid Loss: 8.87  Valid Acc: 0.5479 Valid Pre: 0.5260  Valid Recall: 0.7994 Valid F1: 0.6345  Valid Roc: 0.5357
alpha =  tensor

Evaluation or Testing

In [ ]:
def test():
    best_model = torch.load('/content/drive/MyDrive/GNN/%s.pkl'%(args.conv_name))
    best_model.eval()
    gnn, classifier = best_model
    with torch.no_grad():

        company_emb=gnn.forward(test_risk_data,test_company_attr,test_hete_graph,test_hyp,test_idx,x_test)

        res = classifier.forward(company_emb)

        pred=res.argmax(dim=1)
        ac=acc(test_label,pred)
        pr=pre(test_label,pred)
        re=rec(test_label,pred)
        f=f1(test_label,pred)
        rc=roc(test_label,res[:,1])

        print('Best Test Acc: %.4f Best Test Pre: %.4f Best Test Recall: %.4f Best Test F1: %.4f Best Test ROC: %.4f' % (ac,pr,re,f,rc))
        print(sum(pred))

test()

alpha =  tensor([[0.2223],
        [0.3909],
        [0.5692]])
Outlist len =  3
Outlist 0 len =  3976
Outlist 0 1 len =  12
Outlist 1 len =  3976
Outlist 2 len =  3976
res len =  3976
res 0 len = 12
res 5 len = 12
res 10 len = 12
res 0 =  tensor([ 0.0285,  0.0410,  0.0364, -0.0422, -0.0018, -0.0100,  0.0471,  0.0042,
        -0.0430,  0.0321,  0.0176,  0.0412])
Best Test Acc: 0.5845 Best Test Pre: 0.6887 Best Test Recall: 0.6541 Best Test F1: 0.6710 Best Test ROC: 0.5449
tensor(302)
